In [1]:
import os
import csv
import cv2
from skimage.feature import hog
from sklearn.externals import joblib
import easyocr
import numpy as np
import pandas as pandas

In [2]:
# ucitavanje skupova

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            images.append(img)
    return images 

def load_sets():
    sa_knjigom = load_images_from_folder("skup_podataka/saKnjigom")
    bez_knjige = load_images_from_folder("skup_podataka/bezKnjiga")
    test_skup = load_images_from_folder("skup_podataka/test")
    return sa_knjigom, bez_knjige, test_skup

In [3]:
# odredjivanje da li naslici ima knjiga ili nema

def hog_processing(sa_knjigama, bez_knjiga):
    lista_svih = []
    labele = []
    for slika in sa_knjigama:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        lista_svih.append(features)
        labele.append(1)

    for slika in bez_knjiga:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        lista_svih.append(features)
        labele.append(0)

    clf = joblib.load("pedestrian.pkl")
    clf.fit(lista_svih, labele)
    return clf

def read_resault(ime_fajla):
    resenje = []
    with open(ime_fajla, encoding="utf8", newline='\n') as f:
        lines = csv.reader(f, delimiter=',')
        for row in lines:
            if (row[0] == "br_knjiga"):
                continue
            niz = []
            if len(row) == 2:
                niz = row[1]
                niz = niz[1:-1]
                niz = niz.split(';')
            resenje.append([int(row[0]), niz])
    return resenje

def predict_books(clf, test_skup, informacije):
    resenje = [i[0] != 0 for i in informacije]

    dobijeni_rezultat = []
    ind = 0
    br_tacnih = 0
    for slika in test_skup:
        features = hog(slika, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm="L2")
        listica = [features, features]
        rez = clf.predict(listica)
        dobijeni_rezultat.append(rez[0])
        if (dobijeni_rezultat[ind] == resenje[ind]):
            br_tacnih += 1
        ind += 1
    tacnost = (br_tacnih / len(resenje)) * 100
    tacnost = round(tacnost, 2)
    return tacnost, dobijeni_rezultat

In [4]:
#ocr pomocu biblioteke

def ocr(image):
    reader = easyocr.Reader(['rs_latin'])
    list_of_words = []

    #horizontalne knjige
    output = reader.readtext(image) #[lista_koordinata, rijec_koja_je_detektovana, koliko_je_siguran_da_je_to_ta_rijec]
    for o in output:
        if o[2]>0.5:
            if(len(o[1]) > 1):
                list_of_words.append(o[1])

    #odradi rotaciju za 90 stepeni da detektuje ukoliko ima vertikalnih knjiga
    rot_image = np.rot90(image)
    output = reader.readtext(rot_image) 
    for o in output:
        if o[2]>0.5:
            if(len(o[1]) > 1):
                list_of_words.append(o[1])
        
    return list_of_words

def prepare_ocr_row(row):
    final_list = []
    for elem in row:
        mini_list = elem.split(" ")
        for word in mini_list:
            final_list.append(word.upper())
    return final_list

def calculate_result(ocr_set, csv_list):
    sum_of_all = 0
    num_of_words = 0
    ocr_list = prepare_ocr_row(ocr_set)

    num_of_words = num_of_words + len(csv_list)

    for word in csv_list:
        if word in ocr_list:
            sum_of_all = sum_of_all + 1
    return (sum_of_all/num_of_words) 

In [5]:
# GLAVNA FUNKCIJA

with_books, without_books, test_set = load_sets()
print("Slike sa knjigama:", len(with_books))
print("Slike bez knjiga:", len(without_books))
print("Slike za test:", len(test_set))

print("\nDetekcija da li na slikama ima knjiga ili ne je u toku...")
print("Detekcija moze potrajati oko 10 minuta")
linear_svc = hog_processing(with_books, without_books)
file_info = read_resault('skup_podataka/test_podaci.csv')
accuracy, resault = predict_books(linear_svc, test_set, file_info)
print("\nDetekcija je zavrsena")
print("Tacnost resenja: " + str(accuracy) +"%")

print("\n---------------------------------------------------------------------------------------------")

print("\nDetekcija reci na slikama je u toku...")
print("Detekcija moze potrajati oko 45 minuta")
sum_for_image = 0
image_numer = 0
ind_img = 0
for img in test_set:
    if file_info[ind_img][0] != 0 and resault[ind_img] == 1:
        image_numer += 1
        found_words = ocr(img)
        sum_for_image += calculate_result(found_words, file_info[ind_img][1])
    ind_img += 1
final_avg = round(sum_for_image/image_numer * 100, 2)
print("\nDetekcija je zavrsena")
print("Tacnost resenja: " + str(final_avg) +"%")

Slike sa knjigama: 210
Slike bez knjiga: 210
Slike za test: 150

Detekcija da li na slikama ima knjiga ili ne je u toku...
Detekcija moze potrajati oko 10 minuta


D:\soft_vezbe\sc-2021-siit\env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[LibLinear]

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.



Detekcija je zavrsena
Tacnost resenja: 88.67%

---------------------------------------------------------------------------------------------

Detekcija reci na slikama je u toku...
Detekcija moze potrajati manje od 1 sata


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
CUDA not available - defaulting to CPU. Note: This module is much


Detekcija je zavrsena
Tacnost resenja: 81.2%
